In [1]:
from classication_dataset import ClassificationDataset, get_data_loaders
from architectures.simple_CNN import SimpleCNN, train_model
import torch
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [2]:
for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)

NVIDIA RTX A1000 6GB Laptop GPU


In [3]:
train_loader, val_loader, test_loader = get_data_loaders(
    img_dir="data/PANNUKE/images",
    
    label_file="data/PANNUKE/labels/labels.npy",
    contiguous_ids_file="data/PANNUKE/labels/contiguous_ids.npy",
    
    batch_size=64,
    val_split=0.2,
    test_split=0.1
)

In [4]:
model = SimpleCNN()

In [ ]:
train_model(model, train_loader, val_loader=val_loader, learning_rate=0.001)

Training for epoch 0: 100%|██████████| 23/23 [01:21<00:00,  3.54s/it]


Epoch 1/1000
Train loss: 0.769949337710505
Val loss: 0.6924458265304565


Training for epoch 1: 100%|██████████| 23/23 [01:22<00:00,  3.58s/it]


Epoch 2/1000
Train loss: 0.6879293944524683
Val loss: 0.6701152046521505


Training for epoch 2: 100%|██████████| 23/23 [01:22<00:00,  3.58s/it]


...

Training for epoch 7: 100%|██████████| 23/23 [01:22<00:00,  3.59s/it]


Epoch 8/1000
Train loss: 0.5626941880454188
Val loss: 0.6109986755583021
No progress on val loss was made, patience reduced to 2.


Training for epoch 8: 100%|██████████| 23/23 [01:23<00:00,  3.65s/it]


Epoch 9/1000
Train loss: 0.5746816500373508
Val loss: 0.5530368864536286
No progress on val loss was made, patience reduced to 1.


Training for epoch 9:  65%|██████▌   | 15/23 [00:57<00:30,  3.76s/it]

In [7]:
trained_model_path = "models/simple_CNN_46"
model = SimpleCNN()
model.load_state_dict(torch.load(trained_model_path, weights_only=True))
model.eval()
model.to(torch.device("cuda:0"))

RuntimeError: Error(s) in loading state_dict for SimpleCNN:
	Unexpected key(s) in state_dict: "fc3.weight", "fc3.bias". 
	size mismatch for fc2.weight: copying a param with shape torch.Size([128, 256]) from checkpoint, the shape in current model is torch.Size([2, 256]).
	size mismatch for fc2.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([2]).

In [ ]:

model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
device = torch.device("cuda:0")
all_predictions = []
all_labels = []

    
# Disabling gradient calculation for evaluation
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Get predicted class (the one with the highest score)
        _, predicted = torch.max(outputs, 1)

        # Update the total number of samples and correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        all_labels.append(labels.cpu().numpy())
        all_predictions.append(predicted.cpu().numpy())

# Flatten the lists to arrays
all_predictions = np.concatenate(all_predictions)
all_labels = np.concatenate(all_labels)

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Accuracy on test set: {accuracy:.2f}%')

cm = confusion_matrix(all_labels, all_predictions)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Absent', 'Present'])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

In [8]:
print(model)

SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_run